In [8]:
# All the imports you will need in the whole lab
from skimage.feature import greycomatrix, greycoprops
from skimage import io
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np

%matplotlib qt


In [9]:
def getVerticalProjectionProfile(image):
    image[image == 0] = 1
    image[image == 255] = 0

    vertical_projection = np.sum(image, axis=0)
    return vertical_projection.mean()


In [10]:
def get_glcm_features(gray_scale_img):
    glcm = greycomatrix(gray_scale_img, distances=[1], angles=[np.pi / 2], levels=256)
    contrast = greycoprops(glcm, prop='contrast')
    homogeneity = greycoprops(glcm, prop='homogeneity')
    # energy = greycoprops(glcm, prop='energy')

    return contrast, homogeneity


In [11]:
def get_fname_images_tuple(directory):
    fnames = os.listdir(directory)
    to_return = []
    for fn in fnames:
        if fn[-3:] == 'jpg':
            path = os.path.join(directory, fn)
            gray_scale_image = (rgb2gray(io.imread(path))
                                * 255).astype(np.uint8)
            ###################################################################################
            ###################################################################################
            img_2 = gray_scale_image.copy()
            ###################################################################################
            ###################################################################################
            # if gray_scale_image.mean() < 150:
            #     # THIS MEANS IMAGE IS INVERTED
            #     img_2[gray_scale_image * 255 > 170] = 0
            #     img_2[gray_scale_image * 255 <= 170] = 255
            # else:
            #     img_2[gray_scale_image * 255 > 170] = 255
            #     img_2[gray_scale_image * 255 <= 170] = 0
            ###################################################################################
            ###################################################################################
            to_return.append((fn, img_2))
        else:
            print('This file has been skipped', fn)
    return to_return


In [12]:
def plot_2d_features_with_names(names_fts_list, xlabel, ylabel, zlabel, colors, class_names):
    fig = plt.figure()
    # ax = fig.add_subplot()
    ax = fig.add_subplot(111, projection='3d')

    for class_i in range(len(names_fts_list)):
        x = [names_fts[1][0] for names_fts in names_fts_list[class_i]]
        y = [names_fts[1][1] for names_fts in names_fts_list[class_i]]
        z = [names_fts[2] for names_fts in names_fts_list[class_i]]

        # ax.scatter(x, y, color=colors[class_i], label=class_names[class_i])
        ax.scatter(x, y, z, color=colors[class_i], label=class_names[class_i])

    ax.set_title('Training Data')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_zlabel(zlabel)

    handles, labels = ax.get_legend_handles_labels()
    unique = list(set(labels))
    handles = [handles[labels.index(u)] for u in unique]
    labels = [labels[labels.index(u)] for u in unique]

    ax.legend(handles, labels)

    plt.show()


In [13]:
def extract_features_from_image(number_of_fonts):
    extracted_features = []
    for folder_num in range(1, number_of_fonts + 1):
        folder_path = "../data/raw/" + str(folder_num)
        fn_img_list = get_fname_images_tuple(folder_path)

        fn_ft_list = []
        for fn, img in fn_img_list:
            contrast, homogeneity = get_glcm_features(img)
            fn_ft_list.append((fn, (contrast, homogeneity), getVerticalProjectionProfile(img)))

        extracted_features.append(fn_ft_list)
    return extracted_features


In [14]:
number_of_fonts = 9
colors = ['darkblue', 'green', 'blue', 'yellow',
          'aqua', '#b03060', 'hotpink', 'navajowhite', 'red']
font_names = ['diwani', 'naskh', 'parsi', 'rekaa',
              'thuluth', 'maghribi', 'kufi', 'mohakek', 'Squar-kufic']

font_features = extract_features_from_image(number_of_fonts)

plot_2d_features_with_names(font_features, 'contrast', 'homogeneity',
                            'Vertical Projection Profile Mean', colors, font_names)


ValueError: Unknown projection '3d'